<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>Model Binder اختصاصی</h3>
<ol><li><strong>تبدیل داده‌های پیچیده یا سفارشی</strong>: برای تبدیل داده‌هایی که ساختار خاصی دارند یا از استانداردهای مرسوم (مانند JSON یا فرم‌داده‌ها) پیروی نمی‌کنند.</li><li><strong>اعتبارسنجی داده‌ها</strong>: در صورتی که نیاز به اعتبارسنجی خاصی بر روی داده‌های ورودی داریم و این اعتبارسنجی باید قبل از ایجاد مدل انجام شود.</li><li><strong>پشتیبانی از فرمت‌های داده سفارشی</strong>: مانند فرمت‌های تاریخ و زمان خاص، داده‌های باینری پیچیده یا حتی ترکیب‌های خاص داده‌ها.</li></ol>
</div>

In [ ]:
public class CustomDateModelBinder : IModelBinder
{
    public Task BindModelAsync(ModelBindingContext bindingContext)
    {
        var value = bindingContext.ValueProvider.GetValue(bindingContext.FieldName).FirstValue;

        if (string.IsNullOrEmpty(value))
        {
            bindingContext.Result = ModelBindingResult.Failed();
            return Task.CompletedTask;
        }

        // Trying to parse date in "yyyyMMdd" format
        if (DateTime.TryParseExact(value, "yyyyMMdd", null, System.Globalization.DateTimeStyles.None, out var date))
        {
            bindingContext.Result = ModelBindingResult.Success(date);
        }
        else
        {
            bindingContext.ModelState.AddModelError(bindingContext.FieldName, "Invalid date format. Use 'yyyyMMdd'");
            bindingContext.Result = ModelBindingResult.Failed();
        }

        return Task.CompletedTask;
    }
}

public class MyModel
{
    [ModelBinder(BinderType = typeof(CustomDateModelBinder))]
    public DateTime CustomDate { get; set; }
}

In [ ]:
public class CustomModelBinder : IModelBinder
{
    public Task BindModelAsync(ModelBindingContext bindingContext)
    {
        var customDateValue = bindingContext.ValueProvider.GetValue("customDate").FirstValue;
        var customIntValue = bindingContext.ValueProvider.GetValue("customInt").FirstValue;

        if (DateTime.TryParseExact(customDateValue, "yyyyMMdd", null, System.Globalization.DateTimeStyles.None, out var date) &&
            int.TryParse(customIntValue, out var customInt))
        {
            var model = new MyModel
            {
                CustomDate = date,
                CustomInt = customInt
            };

            bindingContext.Result = ModelBindingResult.Success(model);
        }
        else
        {
            bindingContext.ModelState.AddModelError(bindingContext.ModelName, "Invalid input format.");
            bindingContext.Result = ModelBindingResult.Failed();
        }

        return Task.CompletedTask;
    }
}

[ModelBinder(BinderType = typeof(CustomModelBinder))]
public class MyModel
{
    public DateTime CustomDate { get; set; }
    public int CustomInt { get; set; }
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>ModelBinderProvider چیست؟</h3>
<p><strong>ModelBinderProvider</strong> در واقع یک کلاس است که تصمیم می‌گیرد چه <strong>Model Binder</strong>‌ای برای هر مدل استفاده شود. ASP.NET Core به طور پیش‌فرض چندین <strong>ModelBinderProvider</strong> دارد که مدل بایندرهای مناسب را برای مدل‌های ساده، مجموعه‌ها و ... تعیین می‌کنند. اما با تعریف یک <strong>ModelBinderProvider</strong> اختصاصی، می‌توانیم شرایط خاصی را ایجاد کنیم تا در صورت برخورد با مدل‌ها یا propertyهای خاص، <strong>Custom Model Binder</strong> ما به صورت خودکار اعمال شود.</p>
</div>

In [ ]:

public class CustomModelBinderProvider : IModelBinderProvider
{
    public IModelBinder GetBinder(ModelBinderProviderContext context)
    {
        // بررسی اینکه آیا مدل از نوع MyModel است
        if (context.Metadata.ModelType == typeof(MyModel))
        {
            return new BinderTypeModelBinder(typeof(CustomModelBinder));
        }

        // اگر مدل از نوع MyModel نباشد، به ASP.NET اجازه می‌دهیم بایندر پیش‌فرض را اعمال کند
        return null;
    }
}


//program.cs

services.AddControllerWithViews(options =>
{
    // افزودن CustomModelBinderProvider به ابتدای لیست ModelBinderProviders
    options.ModelBinderProviders.Insert(0, new CustomModelBinderProvider());
});
   

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h3>نحوه استفاده</h3>
<p>حالا، زمانی که <code>MyModel</code> در اکشن کنترلر به عنوان ورودی مدل استفاده شود، نیازی نیست که <code>[ModelBinder]</code> به صورت دستی روی آن قرار بگیرد. <strong>CustomModelBinderProvider</strong> به طور خودکار <strong>CustomModelBinder</strong> را برای <code>MyModel</code> انتخاب و اجرا می‌کند.</p>


<b>یک نمونه خوب از استفاده</b> زمانی که شما میخواهید شناسه یک Object را به سمت کاربر ارسال کنید آن را کد گذاری کنید

</div>